In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# scrapped sites

url_nasa = 'https://mars.nasa.gov/news/'
url_jplimages = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url_weather = 'https://twitter.com/marswxreport?lang=en'
url_facts = 'http://space-facts.com/mars/'
url_himages = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'




In [4]:

response = requests.get(url_nasa)
# nasa_html = response.html
nasa_soup= BeautifulSoup(response.text, 'html.parser')

news_title =nasa_soup.find('div',class_ ='content_title').text
news_p =nasa_soup.find('div',class_ ='rollover_description_inner').text

# Examine the results, then determine element that contains sought info
print(news_title)
print(news_p)
# print(nasa_soup.prettify())



NASA Announces Landing Site for Mars 2020 Rover



After a five-year search, NASA has chosen Jezero Crater as the landing site for its upcoming Mars 2020 rover mission.



In [5]:
response = requests.get(url_jplimages)

jpl_soup= BeautifulSoup(response.text, 'html.parser')
print(jpl_soup.prettify())

<!DOCTYPE html>
<!--[if IE 9]> <html class="no-js ie ie9" lang="en"> <![endif]-->
<!--[if IE 8]> <html class="no-js ie ie8" lang="en"> <![endif]-->
<html>
 <!-- START HEADER: "DEFAULT" -->
 <head>
  <meta charset="utf-8"/>
  <!-- Always force latest IE rendering engine or request Chrome Frame -->
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport"/>
  <title>
   Space Images
  </title>
  <link href="/assets/stylesheets/manifest.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="/assets/stylesheets/print.css" media="print" rel="stylesheet" type="text/css"/>
  <script src="/assets/javascripts/public_manifest.js" type="text/javascript">
  </script>
  <script src="/assets/javascripts/vendor/jquery.fancybox.js" type="text/javascript">
  </script>
  <script src="/assets/javascripts/vendor/jquery.fancybox-thumbs.js" type="text/javascript">
  </script>
 </he

In [6]:
response2 = requests.get(url_jplimages)

jpl_soup= BeautifulSoup(response2.text, 'html.parser')
title =jpl_soup.find('li',class_="slide").find('a',class_="fancybox")['data-fancybox-href']
jpl_img= f'https://www.jpl.nasa.gov{title}'
# title =jpl_soup.a['data-fancybox-href']
# for result in jpl_soup:
#     # Error handling
#     try:
#         # Identify and return title of listing
#         title = result.find('a',class_="button fancybox").text
# #         # Identify and return price of listing
# #         price = result.a.span.text
# #         # Identify and return link to listing
# #         link = result.a['href']
       

#         # Print results only if title, price, and link are available
#         if (title):
            
#             print(title)
            
#     except AttributeError as e:
#         print(e)

print(jpl_img)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22572_hires.jpg


In [7]:
response3 = requests.get(url_weather)

wthr_soup= BeautifulSoup(response3.text, 'html.parser')
mars_weather =wthr_soup.find('p', class_ = "tweet-text").text

print(mars_weather)

Sol 2234 (2018-11-18), high 2C/35F, low -70C/-93F, pressure at 8.57 hPa, daylight 06:25-18:41


In [8]:
# mars facts table
tables = pd.read_html(url_facts)
df = tables[0]
df.columns = ['fact', 'Value']
df.set_index('fact', inplace=True)
df.head()

,Value
fact,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"


In [9]:


hemisphere_image_urls= []

for i in range (1,9,2): 
    
    hemi_dict = {}
    
    browser.visit(url_himages)
    hemi_html = browser.html
    hemi_soup = BeautifulSoup(hemi_html,'html.parser')
    hemi_name_links = hemi_soup.find_all('a', class_ = 'product-item')
    hemi_name = hemi_name_links[i].text.strip('Enhanced')
    detail_links = browser.find_by_css('a.itemLink')
    detail_links[i].click()
    time.sleep(3)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(3)
    browser.windows.current = browser.windows[-1]
    
    hemi_img_html = browser.html
    
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()

    hemi_img_soup = BeautifulSoup(hemi_img_html,'html.parser')
    hemi_img_path = hemi_img_soup.find('img')['src']
    
    print(hemi_name)
    hemi_dict['title'] = hemi_name.strip()
    
    print(hemi_img_path)
    hemi_dict['img_url'] = hemi_img_path
    
    hemisphere_image_urls.append(hemi_dict)


Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


In [10]:
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [36]:
x=df.to_dict()
marstbl_dict=x.pop('Value')

In [37]:
marstbl_dict

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-153 to 20 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

In [38]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Value</th>\n    </tr>\n    <tr>\n      <th>fact</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </t

In [44]:
 mars_data = [hemisphere_image_urls, marstbl_dict,{
        "news_title": news_title,
        "news_p": news_p,
        "jpl_img": jpl_img,
        "mars_weather": mars_weather
       

    }]

In [45]:
mars_data

[[{'title': 'Cerberus Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  {'title': 'Schiaparelli Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  {'title': 'Syrtis Major Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  {'title': 'Valles Marineris Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}],
 {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.52 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-153 to 20 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'},
 {'news_title':

In [53]:

new_dict = {item['title']:item for item in hemisphere_image_urls}

TypeError: list indices must be integers or slices, not dict

In [57]:
 mars_data = [new_dict , marstbl_dict,{
        "news_title": news_title,
        "news_p": news_p,
        "jpl_img": jpl_img,
        "mars_weather": mars_weather
         }]

In [55]:
mars_data


[{'Cerberus Hemisphere': {'title': 'Cerberus Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
  'Schiaparelli Hemisphere': {'title': 'Schiaparelli Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
  'Syrtis Major Hemisphere': {'title': 'Syrtis Major Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
  'Valles Marineris Hemisphere': {'title': 'Valles Marineris Hemisphere',
   'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}},
 {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.52 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-153 t

In [59]:
x=[{k: x[k] for x in d.values() for k in x} for d in mars_data]

TypeError: string indices must be integers